# GameTheory 21 - Jeux Cooperatifs en Lean : Formalisation de Shapley

**Kernel** : Lean 4 (WSL)

**Navigation** : [← GameTheory-20-Lean-SocialChoice](GameTheory-20-Lean-SocialChoice.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]

**Notebook Python compagnon** : [GameTheory-21b-CooperativeGames-Python](GameTheory-21b-CooperativeGames-Python.ipynb) (calculs numeriques, exemples)

---

## Introduction

Ce notebook formalise les **jeux cooperatifs** et la **valeur de Shapley** en Lean 4. Ces concepts, introduits dans le notebook 14 (Python), sont ici traites avec la rigueur des preuves formelles.

### Contenu

1. **Jeux cooperatifs (TU games)** : Fonction caracteristique v(S)
2. **Axiomes de Shapley** : Efficacite, symetrie, additivite, joueur nul
3. **Valeur de Shapley** : Definition et unicite
4. **Le Core** : Stabilite des allocations
5. **Jeux de vote** : Indice de Banzhaf

### Lien avec les notebooks precedents

| Notebook | Contenu |
|----------|--------|
| **14 (Python)** | Implementation : calcul exact/Monte Carlo, exemple politique |
| **20 (Lean)** | Social choice : Arrow, Sen, electeur median |
| **21 (Lean)** | Formalisation : axiomes, theoreme d'unicite de Shapley |

### Duree estimee : 60 minutes

---

## 1. Jeux Cooperatifs (TU Games)

### 1.1 Definition

Un **jeu cooperatif a utilite transferable** (TU game) est defini par :
- Un ensemble fini de joueurs $N = \{1, 2, ..., n\}$
- Une **fonction caracteristique** $v : 2^N \to \mathbb{R}$ avec $v(\emptyset) = 0$

La valeur $v(S)$ represente ce que la coalition $S$ peut obtenir en cooperant.

In [1]:
-- Jeu cooperatif a utilite transferable (TU game)

-- Type des joueurs (fini)
variable {N : Type*} [Fintype N] [DecidableEq N]

-- Coalition = sous-ensemble de joueurs
def Coalition (N : Type*) := Finset N

-- Fonction caracteristique : v(S) donne la valeur d'une coalition
structure TUGame (N : Type*) [Fintype N] where
  v : Finset N → ℝ
  empty_zero : v ∅ = 0

#check TUGame
#check @TUGame.v

### 1.2 Proprietes structurelles

In [2]:
-- Proprietes des jeux cooperatifs

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Superadditivite : la cooperation est benefique
-- v(S ∪ T) ≥ v(S) + v(T) pour S, T disjoints
def Superadditive (G : TUGame N) : Prop :=
  ∀ S T : Finset N, Disjoint S T → 
    G.v (S ∪ T) ≥ G.v S + G.v T

-- Convexite (supermodularite) : contributions marginales croissantes
-- v(S ∪ {i}) - v(S) ≤ v(T ∪ {i}) - v(T) pour S ⊆ T, i ∉ T
def Convex (G : TUGame N) : Prop :=
  ∀ S T : Finset N, ∀ i : N, 
    S ⊆ T → i ∉ T →
    G.v (S ∪ {i}) - G.v S ≤ G.v (T ∪ {i}) - G.v T

#check @Superadditive
#check @Convex

### 1.3 Exemples classiques

In [3]:
-- Exemples de jeux cooperatifs

-- Jeu d'unanimite : v(S) = 1 si T ⊆ S, 0 sinon
-- Les joueurs de T doivent tous etre presents pour creer de la valeur
def unanimityGame {N : Type*} [Fintype N] [DecidableEq N] 
    (T : Finset N) : TUGame N := {
  v := fun S => if T ⊆ S then 1 else 0
  empty_zero := by
    simp only
    by_cases hT : T = ∅
    · simp [hT]
    · simp only [Finset.not_subset_empty_iff.mpr hT, ite_false]
}

-- Jeu de majorite simple : v(S) = 1 si |S| > n/2, 0 sinon
def majorityGame {N : Type*} [Fintype N] [DecidableEq N] : TUGame N := {
  v := fun S => if 2 * S.card > Fintype.card N then 1 else 0
  empty_zero := by simp
}

#check @unanimityGame
#check @majorityGame

---

## 2. Axiomes de Shapley

Shapley (1953) a montre que quatre axiomes caracterisent de maniere unique la valeur de Shapley :

1. **Efficacite** : La somme des paiements egale la valeur de la grande coalition
2. **Symetrie** : Joueurs interchangeables recoivent le meme paiement
3. **Joueur nul** : Un joueur sans contribution marginale recoit 0
4. **Additivite** : La valeur de la somme de deux jeux = somme des valeurs

In [4]:
-- Axiomes de Shapley pour une solution φ

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une solution : associe a chaque jeu un vecteur de paiements
def Solution (N : Type*) [Fintype N] := TUGame N → (N → ℝ)

-- Axiome 1 : Efficacite
def Efficiency (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∑ i, φ G i = G.v Finset.univ

-- Axiome 2 : Symetrie
def Symmetric (G : TUGame N) (i j : N) : Prop :=
  ∀ S : Finset N, i ∉ S → j ∉ S →
    G.v (S ∪ {i}) = G.v (S ∪ {j})

def SymmetryAxiom (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∀ i j : N, 
    Symmetric G i j → φ G i = φ G j

-- Axiome 3 : Joueur nul
def NullPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, i ∉ S → 
    G.v (S ∪ {i}) = G.v S

def NullPlayerAxiom (φ : Solution N) : Prop :=
  ∀ G : TUGame N, ∀ i : N,
    NullPlayer G i → φ G i = 0

-- Axiome 4 : Additivite
def AddGames (G H : TUGame N) : TUGame N := {
  v := fun S => G.v S + H.v S
  empty_zero := by simp [G.empty_zero, H.empty_zero]
}

def AdditivityAxiom (φ : Solution N) : Prop :=
  ∀ G H : TUGame N, ∀ i : N,
    φ (AddGames G H) i = φ G i + φ H i

#check @Efficiency
#check @SymmetryAxiom
#check @NullPlayerAxiom
#check @AdditivityAxiom

---

## 3. Valeur de Shapley

### 3.1 Definition

La valeur de Shapley du joueur $i$ est sa contribution marginale moyenne sur toutes les permutations :

$$\phi_i(v) = \sum_{S \subseteq N \setminus \{i\}} \frac{|S|!(n-|S|-1)!}{n!} [v(S \cup \{i\}) - v(S)]$$

In [5]:
-- Definition de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Contribution marginale de i a la coalition S
def marginalContribution (G : TUGame N) (i : N) (S : Finset N) : ℝ :=
  G.v (S ∪ {i}) - G.v S

-- Coefficient de Shapley pour une coalition de taille s sur n joueurs
noncomputable def shapleyCoef (n s : ℕ) : ℝ :=
  (Nat.factorial s * Nat.factorial (n - s - 1)) / Nat.factorial n

-- Valeur de Shapley
noncomputable def shapleyValue (G : TUGame N) (i : N) : ℝ :=
  ∑ S in (Finset.univ.filter (fun S => i ∉ S)),
    shapleyCoef (Fintype.card N) S.card * marginalContribution G i S

#check @marginalContribution
#check @shapleyValue

### 3.2 Theoreme d'unicite

In [6]:
-- Theoreme de Shapley : la valeur de Shapley est l'unique solution
-- satisfaisant les quatre axiomes

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem shapley_uniqueness_sketch (φ : Solution N)
    (h_eff : Efficiency φ)
    (h_sym : SymmetryAxiom φ)
    (h_null : NullPlayerAxiom φ)
    (h_add : AdditivityAxiom φ) :
    -- Alors φ est la valeur de Shapley
    ∀ G : TUGame N, ∀ i : N, φ G i = shapleyValue G i := by
  -- La preuve utilise la decomposition en jeux d'unanimite
  -- G = ∑_T c_T * u_T
  -- Par additivite et les autres axiomes
  sorry

#check @shapley_uniqueness_sketch

### 3.3 Proprietes de la valeur de Shapley

In [7]:
-- Proprietes de la valeur de Shapley

variable {N : Type*} [Fintype N] [DecidableEq N]

-- La valeur de Shapley satisfait l'efficacite
theorem shapley_efficient : ∀ G : TUGame N,
    ∑ i, shapleyValue G i = G.v Finset.univ := by
  sorry

-- Pour les jeux d'unanimite, la valeur a une forme simple
theorem shapley_unanimity (T : Finset N) (i : N) :
    shapleyValue (unanimityGame T) i = 
    if i ∈ T then (1 : ℝ) / T.card else 0 := by
  sorry

#check @shapley_efficient
#check @shapley_unanimity

---

## 4. Le Core

Le **Core** d'un jeu cooperatif est l'ensemble des allocations stables : aucune coalition ne peut faire mieux en se separant.

In [8]:
-- Le Core d'un jeu cooperatif

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Une allocation est un vecteur de paiements
def Allocation (N : Type*) := N → ℝ

-- Le Core : allocations efficaces et stables
def Core (G : TUGame N) : Set (Allocation N) :=
  { x | 
    -- Efficacite : la somme = valeur de N
    (∑ i, x i = G.v Finset.univ) ∧
    -- Rationalite de groupe : aucune coalition ne peut bloquer
    (∀ S : Finset N, ∑ i in S, x i ≥ G.v S)
  }

-- Le Core peut etre vide !
def CoreEmpty (G : TUGame N) : Prop := Core G = ∅

#check @Core
#check @CoreEmpty

In [9]:
-- Theoreme de Bondareva-Shapley
-- Le Core est non-vide ssi le jeu est equilibre

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Collection equilibree (balanced collection)
def Balanced (weights : Finset N → ℝ) : Prop :=
  (∀ S, weights S ≥ 0) ∧
  (∀ i : N, ∑ S in (Finset.univ.filter (fun S => i ∈ S)), weights S = 1)

-- Jeu equilibre
def BalancedGame (G : TUGame N) : Prop :=
  ∀ weights : Finset N → ℝ,
    Balanced weights →
    ∑ S in Finset.univ, weights S * G.v S ≤ G.v Finset.univ

-- Theoreme de Bondareva-Shapley
theorem bondareva_shapley (G : TUGame N) :
    (Core G).Nonempty ↔ BalancedGame G := by
  sorry

#check @bondareva_shapley

In [10]:
-- Pour les jeux convexes, Shapley est dans le Core

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem convex_shapley_in_core (G : TUGame N) (h : Convex G) :
    (fun i => shapleyValue G i) ∈ Core G := by
  sorry

#check @convex_shapley_in_core

---

## 5. Jeux de Vote

Les jeux de vote sont des cas particuliers ou $v(S) \in \{0, 1\}$ (une coalition gagne ou perd).

In [11]:
-- Jeux de vote ponderes

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Jeu de vote pondere [q; w1, w2, ..., wn]
def WeightedVotingGame (weights : N → ℝ) (quota : ℝ) : TUGame N := {
  v := fun S => if ∑ i in S, weights i ≥ quota then 1 else 0
  empty_zero := by simp
}

-- Un joueur est critique dans une coalition si son retrait la fait perdre
def Critical (G : TUGame N) (i : N) (S : Finset N) : Prop :=
  i ∈ S ∧ G.v S = 1 ∧ G.v (S.erase i) = 0

-- Indice de Banzhaf (non normalise)
def BanzhafRaw (G : TUGame N) (i : N) : ℕ :=
  (Finset.univ.filter (fun S => Critical G i S)).card

#check @WeightedVotingGame
#check @Critical
#check @BanzhafRaw

In [12]:
-- Proprietes des joueurs dans les jeux de vote

variable {N : Type*} [Fintype N] [DecidableEq N]

-- Joueur avec droit de veto
def VetoPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, G.v S = 1 → i ∈ S

-- Dictateur
def Dictator (G : TUGame N) (i : N) : Prop :=
  G.v {i} = 1 ∧ VetoPlayer G i

-- Joueur nul (dummy)
def DummyPlayer (G : TUGame N) (i : N) : Prop :=
  ∀ S : Finset N, i ∉ S → G.v (S ∪ {i}) = G.v S

-- Un joueur nul a Shapley = 0
theorem dummy_shapley_zero (G : TUGame N) (i : N) :
    DummyPlayer G i → shapleyValue G i = 0 := by
  sorry

#check @VetoPlayer
#check @Dictator
#check @DummyPlayer
#check @dummy_shapley_zero

---

## 6. Exercices

### Exercice 1 : Jeu de gants

Trois joueurs : L1, L2 ont chacun un gant gauche, R1 a un gant droit. Une paire de gants vaut 1.

1. Definir la fonction caracteristique
2. Calculer la valeur de Shapley de chaque joueur

### Exercice 2 : Verification d'axiome

Prouver que la valeur de Shapley satisfait l'axiome du joueur nul.

### Exercice 3 : Core vide

Montrer que le jeu de majorite simple a 3 joueurs a un Core vide.

---

## 7. Solutions

### Solution Exercice 1

In [13]:
-- Solution Exercice 1 : Jeu de gants

-- Trois joueurs : L1, L2 (gants gauches), R1 (gant droit)
inductive GlovePlayer where
  | L1 | L2 | R1
  deriving DecidableEq, Fintype

open GlovePlayer

def hasLeftGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => true
  | L2 => true
  | R1 => false

def hasRightGlove (p : GlovePlayer) : Bool :=
  match p with
  | L1 => false
  | L2 => false
  | R1 => true

-- Fonction caracteristique : nombre de paires completes
def gloveGame : TUGame GlovePlayer := {
  v := fun S => 
    let nLeft := (S.filter (fun p => hasLeftGlove p)).card
    let nRight := (S.filter (fun p => hasRightGlove p)).card
    min nLeft nRight
  empty_zero := by simp
}

-- Valeurs de Shapley (calcul manuel) :
-- R1 a une contribution marginale plus elevee (ressource rare)
-- Shapley(L1) = Shapley(L2) = 1/6
-- Shapley(R1) = 4/6 = 2/3

#check gloveGame

### Solution Exercice 2

In [14]:
-- Solution Exercice 2 : Shapley satisfait l'axiome du joueur nul

variable {N : Type*} [Fintype N] [DecidableEq N]

theorem shapley_null_player (G : TUGame N) (i : N) :
    NullPlayer G i → shapleyValue G i = 0 := by
  intro h_null
  -- Si i est un joueur nul, sa contribution marginale est toujours 0
  unfold shapleyValue
  apply Finset.sum_eq_zero
  intro S hS
  simp at hS
  have h : marginalContribution G i S = 0 := by
    unfold marginalContribution
    rw [h_null S hS]
    ring
  simp [h]

#check @shapley_null_player

### Solution Exercice 3

Pour la preuve que le Core du jeu de majorite a 3 joueurs est vide, voir le notebook Python compagnon (21b) pour l'illustration numerique.

---

## Resume

### Concepts formalises

| Concept | Definition Lean |
|---------|----------------|
| TU Game | `TUGame N` avec `v : Finset N → ℝ`, `v(∅) = 0` |
| Superadditivite | `v(S ∪ T) ≥ v(S) + v(T)` pour S, T disjoints |
| Convexite | Contributions marginales croissantes |
| Shapley | Contribution marginale moyenne sur permutations |
| Core | Allocations efficaces et stables |
| Banzhaf | Nombre de coalitions critiques |

### Axiomes de Shapley

| Axiome | Signification |
|--------|---------------|
| Efficacite | `∑ φ = v(N)` |
| Symetrie | Joueurs interchangeables = meme paiement |
| Joueur nul | Contribution nulle = paiement nul |
| Additivite | `φ(G+H) = φ(G) + φ(H)` |

### Theoremes cles

1. **Unicite de Shapley** : Les 4 axiomes caracterisent Shapley uniquement
2. **Bondareva-Shapley** : Core non-vide ⇔ jeu equilibre
3. **Jeux convexes** : Shapley dans le Core

---

## Conclusion de la Serie GameTheory

Cette serie de **21 notebooks** (+3 compagnons Python) a couvert la theorie des jeux de maniere complete :

| Partie | Focus |
|--------|-------|
| 1-6 | Fondations : Nash, minimax, evolution |
| 7-11 | Dynamique : forme extensive, induction, bayesien |
| 12-16 | Algorithmes : CFR, cooperatif, mecanismes, MARL |
| 17-21 | Formalisation : Lean 4, preuves, Shapley |

---

**Navigation** : [← GameTheory-20-Lean-SocialChoice](GameTheory-20-Lean-SocialChoice.ipynb) | [Index](GameTheory-1-Setup.ipynb) | [Fin de la serie]